# 1. 피처 엔지니어링 2: 주소 수집하기

지역마다 호황기인 산업이 있고, 침체기인 산업이 존재한다. 따라서 우리는 주소를 수집하여 지역별 산업 현황을 살펴보고,

지역에 따른 산업군 분포를 파악하여 새로운 파생변수를 만들기 위해 주소 데이터를 수집한다.

기업의 정보를 검색할 수 있는 나이스 비즈인포에서 selenium을 통해 크롤링하여 주소 데이터를 수집하고자 한다.

### 병렬 처리를 적용한 크롤링 코드

- 주소를 수집해야할 데이터의 개수가 약 3만개로, 일반적인 크롤링을 시도하면 약 7일이 소요된다.
- 7일 동안 크롤링을 하기에는 무리가 있기 때문에 병렬 처리를 시도했다.
    - 7일 -> 11시간으로 크롤링 시간을 대폭 단축할 수 있었다.
- 현재 사용하는 CPU의 프로세서 개수는 12개 이므로, 데이터를 12개의 구간으로 나누어서 진행하였다.
- 산업코드 크롤링을 할 때와 동일하게 사업자등록번호, 기업명으로 검색해도 나오지 않는 지점인 기업들은 수작업으로 찾아서 데이터를 수집했다.

```python
# 필요한 라이브러리 불러오기
import json
import pandas as pd

import multiprocessing
from multiprocessing import Pool

from selenium import webdriver              
from selenium.webdriver import ActionChains  
from selenium.webdriver.common.by import By  
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException

def get_data(data):
    """
    나이스 비즈인포에서 사업자등록번호 or 기업명을 검색하여 주소를 수집하는 함수입니다.
    -------------------------------------------
    input = data(접속할 url, 검색할 데이터(list))
    -------------------------------------------
    """
    result_dict = [{'사업자등록번호':[], '기업명':[], '주소':[]}, {'사업자등록번호':[], '기업명':[]}] # 크롤링한 결과를 저장할 변수 선언
    # 예외처리를 위해 try ~ except문 사용
    try:
        # selenium으로 크롤링하기 위해 크롬 드라이버 불러오기
        dr = webdriver.Chrome()
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        dr.set_window_size(1000, 1000)                   
        dr.get(url=data[0]) 
        time.sleep(2)
        
        # 사업자등록번호, 기업명으로 검색해서 크롤링하기
        for cor_number, cor_name in zip(data[1], data[2]):
            # 01. 검색창에 사업자등록번호를 입력하고 검색버튼 누르기 (sleep=5)
            search_box_biz_info = dr.find_element(By.XPATH, "/html/body/div[2]/div/ul/li[1]/div/form/input")     # 비즈인포 검색창
            search_button_biz_info = dr.find_element(By.XPATH, '/html/body/div[2]/div/ul/li[1]/div/form/button') # 비즈인포 검색 버튼
            time.sleep(2)

            act = ActionChains(dr)
            act.send_keys_to_element(search_box_biz_info, cor_name).click(search_button_biz_info).perform() # 사업자등록번호 입력, 버튼 클릭 수행
            time.sleep(5)

            # 예외처리를 위해 try ~ except문 사용
            try:
                # 02. 검색해서 나온 결과에서 주소 가져오기 (sleep=3)
                click_result = dr.find_element(By.XPATH, '/html/body/div[3]/div/div[2]/table/tbody/tr[2]/td/div/ul/li[1]')
                time.sleep(2)

                result_dict[0]["사업자등록번호"].append(cor_number)
                result_dict[0]["기업명"].append(cor_name) 
                result_dict[0]["주소"].append(click_result.text)
                
            except NoSuchElementException as e:
                result_dict[1]["사업자등록번호"].append(cor_number)
                result_dict[1]["기업명"].append(cor_name)
                print(f'{cor_number} {cor_name}의 정보가 없습니다.')
            
            except UnexpectedAlertPresentException as e:
                result_dict[1]["사업자등록번호"].append(cor_number)
                result_dict[1]["기업명"].append(cor_name)
                print(f'{cor_number} {cor_name}의 정보가 없습니다.')

            # 중간저장
            filename = f'file_{data[1][0]}.json'
            with open(f'./custom_data/address_json_name_non2/{filename}','w') as f:
                json.dump(result_dict, f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(e)

    finally:
        dr.close()
        dr.quit()
        
if __name__ == '__main__':
    df = pd.read_csv('./crawling_data.csv', encoding='cp949')
    df.drop_duplicates(subset='사업자등록번호', inplace=True)  # 결산으로 인한 중복 데이터가 존재하므로 사업자등록번호를 기준으로 중복제거
    num_list = []
    name_list = []
    numbers = df['사업자등록번호'].values.tolist()
    names = df['기업명'].values.tolist()
    start = 0
    end = 2975
    # 데이터를 12개의 구간으로 나누기
    for i in range(0, 12):
        if i != 11:
            num_list.append(numbers[start:end])
            name_list.append(names[start:end])
            start = end
            end += 2975
        else:
            num_list.append(numbers[start:])
            name_list.append(names[start:])

    # 병렬 처리를 위한 CPU 나누기 및 데이터 만들기
    cpu_count = multiprocessing.cpu_count()
    url = 'https://www.nicebizinfo.com/cm/CM0100M001GE.nice'
    urls_list = [url] * cpu_count
    total = []
    for i, j, k in zip(urls_list, num_list, name_list):
        total.append([i, j, k])
    p = Pool(processes=cpu_count)
    p.map(get_data, total)
    
```

# 2. 수집한 주소 합치기

In [1]:
import json
import os

# 사업자등록번호로 검색한 결과물

path = "./custom_data/address_json/"
file_list = os.listdir(path)

data = []
for filename in file_list:
    with open(path+filename) as file_1:
	    data.append(json.load(file_1))

finish = data[0][0]
for i in range(1, len(data)):
    finish['사업자등록번호'].extend(data[i][0]['사업자등록번호'])
    finish['주소'].extend(data[i][0]['주소'])

retry = data[0][1]
for i in range(1, len(data)):
    retry['사업자등록번호'].extend(data[i][1]['사업자등록번호'])
    retry['기업명'].extend(data[i][0]['기업명'])

In [2]:
# 중간에 세션이 끊어져 누락된 데이터들 재검색(약 1000개)

path = "./custom_data/address_json_no/"
file_list = os.listdir(path)

data2 = []
for filename in file_list:
    with open(f'./custom_data/address_json_no/{filename}') as file_1:
	    data2.append(json.load(file_1))

for i in range(0, len(data2)):
    finish['사업자등록번호'].extend(data2[i][0]['사업자등록번호'])
    finish['주소'].extend(data2[i][0]['주소'])

for i in range(0, len(data2)):
    retry['사업자등록번호'].extend(data2[i][1]['사업자등록번호'])
    retry['기업명'].extend(data2[i][0]['기업명'])

In [3]:
# 사업자등록번호로 나오지 않아서 기업명으로 검색한 결과물

path = "./custom_data/address_json_name/"
file_list = os.listdir(path)

data3 = []
for filename in file_list:
    with open(f'./custom_data/address_json_name/{filename}') as file_1:
	    data3.append(json.load(file_1))
        
for i in range(0, len(data3)):
    finish['사업자등록번호'].extend(data3[i][0]['사업자등록번호'])
    finish['주소'].extend(data3[i][0]['주소'])

retry2 = data3[0][1]
for i in range(1, len(data3)):
    retry2['사업자등록번호'].extend(data3[i][1]['사업자등록번호'])
    retry2['기업명'].extend(data3[i][0]['기업명'])

In [4]:
# 중간에 세션이 끊어져 누락된 데이터들 재검색(약 100개)

path = "./custom_data/address_json_name_non/"
file_list = os.listdir(path)

data3 = []
for filename in file_list:
    with open(f'./custom_data/address_json_name_non/{filename}') as file_1:
	    data3.append(json.load(file_1))
        
for i in range(0, len(data3)):
    finish['사업자등록번호'].extend(data3[i][0]['사업자등록번호'])
    finish['주소'].extend(data3[i][0]['주소'])

retry2 = data3[0][1]
for i in range(1, len(data3)):
    retry2['사업자등록번호'].extend(data3[i][1]['사업자등록번호'])
    retry2['기업명'].extend(data3[i][0]['기업명'])

## 2-1. 수작업으로 수집한 주소 데이터 형식 바꾸기

In [26]:
# 딕셔너리 형태인데, key가 기업명이므로, 사업자등록번호로 key를 바꿔준다.

# 바꿔질 데이터
with open(f'./custom_data/search_address.json', encoding='utf-8') as file_1:
	search_address = json.load(file_1)

# key값을 바꾸기 위해 사용될 데이터
with open(f'./custom_data/search_address.json', encoding='utf-8') as file_1:
	revise_address = json.load(file_1)

In [34]:
import pandas as pd

df = pd.read_csv('./custom_data/basic_finance_data.csv', encoding='cp949')
df.drop_duplicates(subset='사업자등록번호', inplace=True)

In [35]:
# 사업자번호추출하기
for search_result in search_address:
    cor_numbers = df.loc[df['기업명']==search_result, '사업자등록번호'].values[0]
    revise_address[search_result] = cor_numbers

In [36]:
# 추출한 사업자등록번호로 key 값 바꾸기
fianl_address = dict((revise_address[key], value) for (key, value) in search_address.items())

# 3. 원본 데이터에 주소 데이터 합치기

In [38]:
# 중복이 제거되지 않은 데이터로 다시 불러오기
df = pd.read_csv('./custom_data/basic_finance_data.csv', encoding='cp949')

In [39]:
# 크롤링으로 수집한 데이터들 합치기
for cor_number, cor_address in zip(finish['사업자등록번호'], finish['주소']):
    df.loc[df['사업자등록번호']==cor_number, "주소"] = cor_address

In [40]:
# 수작업으로 수집한 데이터들 합치기
for cor_number in fianl_address:
    df.loc[df['사업자등록번호']==cor_number, "주소"] = fianl_address[cor_number]

# 4. 내보내기

In [41]:
df.to_csv('./custom_data/feature_engineering_address.csv', encoding='utf-8', index=False)